# 2022年度数据分析报告 ｜杰然不同之GR


# 本报告由公众号【杰然不同之GR】编写，整理并发布，仅作个人学习使用，请勿用于任何商业用途，转载及其他形式合作请与我联系 

简易目录

1.数据预处理
   
1.1 数据合并和一些纠错

1.2 新特征生成

1.2.1) 根据date生成月日还有星期

1.2.2) 根据start time 和end time 生成时和分

1.2.3) 根据start和end生成中间时间

1.2.4) 根据事项的中间时间确定事项做的时候处于一天的时段

1.2.5) 根据时长大小分为长中短

1.2.6) 修改一些前后差别不大的事件

1.2.7) 统一事项与属性的对应关系 

2.数据可视化

2.1 变量分类

2.1.1) 日期类变量: date

2.1.2) 数值类变量: duration, phone 

2.1.3) 类别变量: event, year, month, day, weekday, mid_hour, day_period, week_order, duration_attr, attr, 

2.2 单变量分析

2.2.1) 数值变量的直方图

2.2.2) 类别变量的饼图，柱形图

2.2.3) 专注力分析

2.2.4) 各个月份，各年, 各小时的event关键词词云图

2.3 双变量分析

2.3.1) PyCatFlow图

2.3.2) 单个类别变量 vs 数值 sns.stripplot  sns.swarmplot  sns.boxplot   sns.violinplot  

2.3.3) 两个类别变量 VS 数值 sns.countplot  sns.barplot  sns.factorplot  sns.pointplot 

2.4 衍生数据可视化

2.4.1) 日度汇总各类时间总时长

2.4.2) 每天的第一件和最后一件事

2.4.3) 每天的第一件和最后一件事的时间

2.4.4) 每天做的事项数目的统计

2.4.5) 每天做事项先后编号并统计

2.4.6) 各事项关于日期求和的透视表

2.4.7) 每年, 月, 星期的事项关键词词云图

2.4.8) 每天事项的对应先后以及月度关联的网络图

2.5 一键式EDA

3.数据分析

3.1 方差分析每月或是每年的事项是否有差异

3.1.1) 原始事项时长数据的方差分析

3.1.2) 衍生数据的方差分析

3.2 LSTM预测前后事项

3.2.1) 建立字符索引

3.2.2) 建立LSTM模型

3.2.3) 模型训练

3.3 时长数据的简单聚类

3.4 关于日期的缺失值处理

3.5 关于日期的傅里叶分解求周期

3.6 关于日期的传统时间序列分析

3.6.1) ARIMA分析

3.6.2) GARCH分析

3.6.3) VAR分析

3.7 关于日期的NeuralProphet 时间序列

4.数据挖掘

4.1 通过生成聚合特征做日度时长数据的特征工程

4.2 类别变量的one-hot encoding

4.3 利用boruta筛选特征

4.4 利用PCA降维

4.5 利用optuna优化参数

4.6 stacking 融合

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import jieba
import statsmodels.tsa.stattools as st
import warnings
import lightgbm as lgb
import xgboost as xgb
import networkx as nx 
import pycatflow as pcf
import statsmodels.api as sm
import statsmodels
from collections import Counter
import optuna
import pylab as pl 
import matplotlib.cm as cm 
import torch
import random
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.stats import kstest, shapiro
from optuna.samplers import TPESampler
from gensim.corpora import Dictionary 
from gensim.models import TfidfModel
from wordcloud import WordCloud
from scipy import stats, signal, interpolate
from arch import arch_model
from boruta import BorutaPy
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa import stattools
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARMA
from statsmodels.stats.diagnostic import acorr_ljungbox
from sklearn.preprocessing import StandardScaler 
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from autoviz.AutoViz_Class import AutoViz_Class
from tqdm import tqdm
from neuralprophet import NeuralProphet
from PIL import Image, ImageDraw, ImageFont
from matplotlib import rcParams
from torch import nn
from mpl_toolkits.mplot3d import Axes3D
from utils import part1, part2, part3, part4, funcs
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import GroupKFold, KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

config = {
    "font.family":'serif',
    "font.size": 40,
    'font.weight':'bold',
    "mathtext.fontset":'stix',
    'figure.titleweight': 'bold',
    'axes.titleweight': 'bold',
    'axes.labelweight': 'bold',
    "font.serif": ['STSong'],
    'axes.unicode_minus':False
}
rcParams.update(config)
path = 'D:/打卡/2022/年度总结'
warnings.filterwarnings("ignore")
%matplotlib inline

# 1.数据预处理

## 1.1 数据合并和一些纠错

In [ ]:
# 循环读入每天的数据
df = pd.DataFrame()
for month in [f'0{n}' for n in range(1, 10)] + ['10', '11', '12']:
    for day in [f'0{n}' for n in range(1, 10)] + [str(n) for n in range(10, 32)]:
        file = f'2022-{month}-{day}打卡记录.xlsx'
        phone = f'2022-{month}-{day}打卡总结.xlsx'
        try:
            temp = pd.read_excel(file)
            temp['phone'] = pd.read_excel(phone).loc[0, '手机时间']
            df = pd.concat([df, temp], axis=0)
        except:
            continue

In [ ]:
df

In [ ]:
# 一些纠错
corrects = {
    '口音':'口语', 
    '策划摄影':'摄影',
    '买食物':'购物', 
    '驾校体检':'咨询驾校', 
    '洗澡':'洗漱',
    '法律咨询':'咨询',
    '生日准备':'购物',
    '买蛋糕':'购物',
    '蒸饭':'煮饭',
    '准备礼物':'购物',
    '摄影策划':'摄影',
    '拿外卖':'拿快递',
    '买东西':'购物'
}

df['event'] = df['event'].apply(lambda x: corrects.get(x) if corrects.get(x) is not None else x)
df.to_excel('2022汇总.xlsx', index=False)

## 1.2  新特征生成

In [ ]:
df = pd.concat(
    [
        pd.read_excel(f'{n}汇总.xlsx')['date	event	start	end	duration	attr	phone'.split()]
        for n in [2019, 2020, 2021, 2022]
    ], axis=0
)

### 1.2.1 根据date生成月日还有星期

In [ ]:
# 1.2.1 根据date生成月日还有星期
df['event'] = df['event'].str.strip()
df['year'] = df['date'].astype(str).apply(lambda x:str(x.split('-')[0]))
df['month'] = df['date'].astype(str).apply(lambda x:str(x.split('-')[1]))
df['day'] = df['date'].astype(str).apply(lambda x:str(x.split('-')[2]))
df['weekday'] = pd.to_datetime(df['date']).dt.weekday + 1

### 1.2.2 根据start time和end time生成时和分

In [ ]:
# 1.2.2 根据start time和end time生成时和分
df['start_hour'] = df['start'].astype(str).apply(lambda x:str(x.split(':')[0]))
df['end_hour'] = df['end'].astype(str).apply(lambda x:str(x.split(':')[0]))

### 1.2.3 根据start和end生成中间时间

In [ ]:
# 1.2.3 根据start和end生成中间时间
df['mid_time'] = df.apply(lambda x: part1.get_mid(x['start'], x['duration']), axis=1)

### 1.2.4 根据事项的中间时间确定事项做的时候处于一天的时段

In [ ]:
# 1.2.4 根据事项的中间时间确定事项做的时候处于一天的时段
df['mid_hour'] = df['mid_time'].astype(str).apply(lambda x:str(x.split(':')[0]))
df['day_period'] = df['mid_hour'].map(part1.get_period)
df['week_order'] = df['date'].map(lambda x: pd.Period(x, freq='W'))

### 1.2.5 根据时长大小分为长中短

In [ ]:
df['duration_attr'] = df['duration'].map(part1.get_duration)
df.tail()

### 1.2.6 修改一些前后差别不大的事件

In [ ]:
# 1.2.6 修改一些前后差别不大的事件
df['event'] = df['event'].map(part1.correct_events)

### 1.2.7 事项与属性的对应

In [ ]:
# 1.2.7 事项与属性的对应
# 各别事项前后不统一 需要统一修正 修正成该事项出现最多的属性
events = df['event'].unique()
event2attr = {}
for e in events:
    attr = df[df['event'] == e]['attr'].value_counts().index[0]
    event2attr[e] = attr

df['attr'] = df['event'].map(event2attr)
df.to_excel('4年汇总.xlsx', index=False)

In [ ]:
# 检查是否还存在事项的属性不唯一的情况
df[['event', 'attr']].drop_duplicates().to_excel('事项性质对应表.xlsx', index=False)
df[['event', 'attr']].drop_duplicates()['event'].value_counts()

# 2.数据可视化

## 2.1 变量分类

In [ ]:
date_vars = ['date']
years = [2022]#range(2019, 2023)

numeric_vars = [
    'duration', 'phone'
]

cate_vars = [
    'event', 'year', 'month', 'day', 
    'weekday', 'mid_hour', 'day_period', 'week_order', 'duration_attr', 'attr'
]

## 2.2 单变量分析

### 2.2.1) 数值变量的直方图

In [ ]:
# 2.2.1) 数值变量的直方图
part = '2.2.1'
for col in numeric_vars:
    for year in years:
        part2.numeric_hist(df, col, year, part, path)

### 2.2.2) 类别变量的饼图

In [ ]:
# 2.2.2) 类别变量的饼图
# 类别变量饼图
part = '2.2.2'

for col in cate_vars:
    if col == 'year':
        continue
    for year in years:
        part2.plot_pie(df, col, 15, part, path, year)

### 2.2.3) 专注力分析

In [ ]:
# 2.2.3) 专注力分析
part = '2.2.3'
title = f'{part} 有效时间在不同年和时长段的柱形图'
temp = df[df['attr'] == '有效时间'].groupby(['year', 'duration_attr'])['date'].\
    count().unstack()
temp = temp / temp.sum(axis=0)  #计算频率
temp.plot(
        kind='bar', 
        figsize=(20, 10), 
    )
plt.title(title, fontsize=20)
plt.xlabel('Year', fontsize=20)
plt.ylabel('Frequency', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(fontsize=20)
file = os.path.join(path, f'{title}.png')
plt.savefig(file, dpi=100)

In [ ]:
part2.concentrate(df, 2022, part, path)

In [ ]:
part2.concentrate(df, 2021, part, path)

In [ ]:
part2.concentrate(df, 2020, part, path)

In [ ]:
part2.concentrate(df, 2019, part, path)

## 2.3 双变量分析

### 2.3.1) PyCatFlow图

In [ ]:
# 2.3.1) PyCatFlow图
column_order = { 
    date: i + 1 for i, date in enumerate(df['date'].unique()) 
}
df['column_order'] = df['date'].map(column_order)
translate = pd.read_csv('event.csv', encoding='utf-8')
chi = translate['Chinese'].values
eng = translate['English'].dropna().values
translates = dict(zip(chi, eng))

translates

In [ ]:
df['eng_event'] = df['event'].map(translates)

df['eng_attr'] = df['attr'].map({
    '有效时间':'effective',
    '浪费时间':'wasteful',
    '必要时间':'necessary',
})

temp = df[df['year'] == '2022']
temp[['eng_event', 'eng_attr', 'date', 'column_order']].\
to_csv('2022.csv', encoding='gbk', index=False)

In [ ]:
# Loading and parsing data:
data = pcf.read_file(
    "2022.csv", 
    columns="date", 
    nodes="eng_event", 
    categories="eng_attr", 
    column_order="column_order"
)

# Generating the visualization
viz = pcf.visualize(
    data,
    spacing=100,
    width=88000,
    maxValue=20,
    minValue=2
)
viz

### 2.3.2) 单个类别变量 vs 数值

In [ ]:
# 2.3.2) 单个类别变量 vs 数值
# 类别变量 VS 数值变量
part = '2.3.2'
for num in numeric_vars:
    for cate in cate_vars:
        part2.plot_num_vs_cate(df, num, cate, path, part)

In [ ]:
for num in numeric_vars:
    for cate in cate_vars:
        part2.plot_box(df, num, cate, path, part)

### 2.3.3) 两个类别变量 VS 数值

In [ ]:
# 2.3.3) 两个类别变量 VS 数值
part = '2.3.3'
for idx, v in enumerate(cate_vars):
    if idx == len(cate_vars) - 1:
        break
    for k in cate_vars[idx + 1:]:
        part2.plot_count_plot(df, v, k, path, part)

## 2.4 衍生数据可视化

### 2.4.1) 日度汇总各类时间总时长

In [ ]:
# 2.4.1) 日度汇总各类时间总时长
part = '2.4.1'
daily_attr = df.groupby(['date', 'attr'])['duration'].\
    agg('sum').unstack().reset_index()

daily_attr = pd.merge(
    daily_attr, 
    df[['date', 'phone']].drop_duplicates(), 
    on='date'
).rename(columns={'phone':'手机时间'})

daily_attr

In [ ]:
daily_attr.describe()

In [ ]:
daily_attr[daily_attr['date'] >= '2022-01-01'].describe()

In [ ]:
title = f'{part} 日度汇总各类时间总时长'
daily_attr.set_index('date').plot(
    kind='line', figsize=(20, 10)
)

plt.title(title, fontsize=20)
plt.xlabel('Date', fontsize=20)
plt.ylabel('Sum of daily duration', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.legend(loc=0, fontsize=20)

file = os.path.join(path, f'{title}.png')
plt.savefig(file, dpi=100)

In [ ]:
# 生成新数据
daily_attr['weekday'] = pd.to_datetime(daily_attr['date']).dt.weekday + 1
daily_attr['month'] = daily_attr['date'].apply(lambda x:str(x).split('-')[1])
daily_attr['day'] = daily_attr['date'].apply(lambda x:str(x).split('-')[2])
daily_attr['week_order'] = daily_attr['date'].map(lambda x: pd.Period(x, freq='W'))
daily_attr['day_of_year'] = pd.to_datetime(daily_attr['date']).dt.day_of_year
daily_attr['year'] = pd.to_datetime(daily_attr['date']).dt.year

In [ ]:
# 每年各类时间折线图对比
for col in '必要时间	有效时间	浪费时间	手机时间'.split():
    part2.line_compare(daily_attr, col, part, path)

In [ ]:
# 插值
daily_attr_copy = daily_attr.copy()
daily_attr_copy = daily_attr_copy.rename(
    columns={
        '必要时间': 'necessary time',	
        '有效时间': 'effective time',	
        '浪费时间': 'waste time',	
        '手机时间': 'phone time',
    }
)
daily_attr_copy = daily_attr_copy.dropna()
daily_attr_copy = daily_attr_copy[daily_attr_copy['date'] >= '2022-01-01']

In [ ]:
x, y, z = 'waste time', 'necessary time', 'effective time'
part2.daily_interpolate(daily_attr_copy, x, y, z, path, part, n=400)

In [ ]:
x, y, z = 'phone time', 'necessary time', 'effective time'
part2.daily_interpolate(daily_attr_copy, x, y, z, path, part)

In [ ]:
# 4年各类时间在月，日，星期的对比
for c in ['weekday', 'day_of_year', 'month']:
    for tt in '必要时间	有效时间	浪费时间	手机时间'.split():
        for fun in [part2.get_max_sum_year, part2.get_max_mean_year]:
            part2.plot_max_year(daily_attr, c, tt, fun, part, path)

### 2.4.2) 每天的第一件和最后一件事

In [ ]:
# 2.4.2) 每天的第一件和最后一件事
part = '2.4.2'
first_last_event = pd.DataFrame()

# 每天做的第一件事
first_last_event['first_event'] = df[['date', 'event']].\
    groupby(['date'])['event'].first()

# 每天做的最后一件事
first_last_event['last_event'] = df[['date', 'event']].\
    groupby(['date'])['event'].last()

    
first_last_event = first_last_event.reset_index()
first_last_event

In [ ]:
df.groupby(['year'])['event'].value_counts().unstack()

In [ ]:
df.groupby(['year', 'event'])['duration'].sum().reset_index()

In [ ]:
# 整体的饼图
part2.plot_pie(first_last_event, 'first_event', 15, part, path)

In [ ]:
# 2019饼图
part2.plot_pie(
    first_last_event[first_last_event['date'] < '2020-01-01'], 
    'first_event', 
    15, 
    part, 
    path,
    year=2019
)
part2.plot_pie(
    first_last_event[first_last_event['date'] < '2020-01-01'], 
    'last_event', 
    15, 
    part, 
    path,
    year=2019
)

In [ ]:
# 2020饼图
temp = first_last_event[
    (first_last_event['date'] < '2021-01-01') & (first_last_event['date'] >= '2020-01-01')
]
part2.plot_pie(
    temp, 
    'first_event', 
    15, 
    part, 
    path,
    year=2020
)
part2.plot_pie(
    temp, 
    'last_event', 
    5, 
    part, 
    path,
    year=2020
)

In [ ]:
# 2021饼图
temp = first_last_event[
    (first_last_event['date'] < '2022-01-01') & (first_last_event['date'] >= '2021-01-01')
]
part2.plot_pie(
    temp, 
    'first_event', 
    15, 
    part, 
    path,
    year=2021
)
part2.plot_pie(
    temp, 
    'last_event', 
    5, 
    part, 
    path,
    year=2021
)

In [ ]:
# 2022饼图
temp = first_last_event[
    (first_last_event['date'] < '2023-01-01') & (first_last_event['date'] >= '2022-01-01')
]
part2.plot_pie(
    temp, 
    'first_event', 
    15, 
    part, 
    path,
    year=2022
)
part2.plot_pie(
    temp, 
    'last_event', 
    5, 
    part, 
    path,
    year=2022
)

### 2.4.3) 每天的第一件和最后一件事的时间

In [ ]:
# 2.4.3) 每天的第一件和最后一件事的时间
part = '2.4.3'
first_last_time = pd.merge(
    df.groupby(['date'])['start', 'start_hour'].first(), 
    df.groupby(['date'])['end', 'end_hour'].last(), 
    on='date'
).reset_index().rename(columns={
    'start':'first_start_time', 
    'start_hour':'first_start_hour',
    'end':'last_end_time', 
    'end_hour':'last_end_hour'
})

first_last_time

In [ ]:
# 2019饼图
temp = first_last_time[first_last_time['date'] < '2020-01-01']
part2.plot_pie(
    temp, 
    'first_start_hour', 
    15, 
    part, 
    path,
    year=2019
)
part2.plot_pie(
    temp, 
    'last_end_hour', 
    5, 
    part, 
    path,
    year=2019
)

In [ ]:
# 2020饼图
temp = first_last_time[
    (first_last_time['date'] < '2021-01-01') & (first_last_time['date'] >= '2020-01-01')
]
part2.plot_pie(
    temp, 
    'first_start_hour', 
    15, 
    part, 
    path,
    year=2020
)
part2.plot_pie(
    temp, 
    'last_end_hour', 
    5, 
    part, 
    path,
    year=2020
)

In [ ]:
# 2021饼图
temp = first_last_time[
    (first_last_time['date'] < '2022-01-01') & (first_last_time['date'] >= '2021-01-01')
]
part2.plot_pie(
    temp, 
    'first_start_hour', 
    15, 
    part, 
    path,
    year=2021
)
part2.plot_pie(
    temp, 
    'last_end_hour', 
    5, 
    part, 
    path,
    year=2021
)

In [ ]:
# 2022饼图
temp = first_last_time[
    (first_last_time['date'] < '2023-01-01') & (first_last_time['date'] >= '2022-01-01')
]
part2.plot_pie(
    temp, 
    'first_start_hour', 
    15, 
    part, 
    path,
    year=2022
)
part2.plot_pie(
    temp, 
    'last_end_hour', 
    5, 
    part, 
    path,
    year=2022
)

### 2.4.4) 每天做的事项数目的统计

In [ ]:
# 2.4.4) 每天做的事项数目的统计
part = '2.4.4'
df.groupby('date')['event'].count().\
    reset_index().\
    sort_values(by='event', ascending=False)

In [ ]:
title = f'{part} 每天做的事项数目的统计'
df.groupby('date')['event'].count().plot(
    figsize=(20, 10)
)
plt.title(title, fontsize=20)
plt.xlabel('Date', fontsize=20)
plt.ylabel('Count', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

file = os.path.join(path, f'{title}.png')
plt.savefig(file, dpi=100)

In [ ]:
title = f'{part} 每天做的事项数目的统计直方图'
df.groupby('date')['event'].count().plot(
    kind='hist', figsize=(20, 10)
)

plt.title(title, fontsize=20)
plt.xlabel('Count', fontsize=20)
plt.ylabel('Frequency', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

file = os.path.join(path, f'{title}.png')
plt.savefig(file, dpi=100)

### 2.4.5) 各年，每天，每月做得最多的事以及各年的相似度

In [ ]:
# 各年
df.groupby(['year'])[['event']].agg(
    lambda x:','.join(x[0] for x in Counter(x).most_common(10))
)

In [ ]:
# 每月做的次数最多的事
df.groupby(['month'])[['event']].agg(
    lambda x:','.join(x[0] for x in Counter(x).most_common(10))
)

In [ ]:
# 各天做的次数最多的事
df.groupby(['weekday'])[['event']].agg(
    lambda x:','.join(x[0] for x in Counter(x).most_common(10))
)

In [ ]:
# 各年做的总时间最多的事
df.groupby(['year', 'event'])[['duration']].sum().\
    reset_index().groupby('year')[['event', 'duration']].apply(
        lambda x:x[x['duration'].isin(x['duration'].nlargest(3))].\
            sort_values(by='duration', ascending=False)
    )

In [ ]:
# 各月做的总时间最多的事
df.groupby(['month', 'event'])[['duration']].sum().\
    reset_index().groupby('month')[['event', 'duration']].apply(
        lambda x:x[x['duration'].isin(x['duration'].nlargest(5))].\
            sort_values(by='duration', ascending=False)
    )

In [ ]:
# 各天做的总时间最多的事
df.groupby(['weekday', 'event'])[['duration']].sum().\
    reset_index().groupby('weekday')[['event', 'duration']].apply(
        lambda x:x[x['duration'].isin(x['duration'].nlargest(5))].\
            sort_values(by='duration', ascending=False)
    )

In [ ]:
# 计算各年的余弦相似度
for col in ['year', 'month', 'weekday']:
    part2.plot_sim_heatmap(df, col, part, path)

### 2.4.6) 每天做事项先后编号并统计

In [ ]:
# 2.4.6) 每天做事项先后编号并统计
part = '2.4.6'
def get_order(alist):
    return range(1, 1 + len(alist))

daily_event_order = df.groupby('date')['event'].agg(get_order)
daily_event_order = pd.concat(
    [daily_event_order.explode().reset_index().rename(columns={'event':'order'}), 
    df['event'].reset_index()], axis=1
)[['date', 'event', 'order']]
daily_event_order['order'] = daily_event_order['order'].astype(int)
daily_event_order

In [ ]:
# 每个事项平均处于每天的第几件事
event_mean_order = daily_event_order[
    (daily_event_order['date'] < '2023-01-01') & 
    (daily_event_order['date'] >= '2022-01-01')
].groupby('event')['order'].mean()

title = f'{part} 各事项平均顺序的统计柱形图'
event_mean_order.sort_values().plot(
    kind='bar', figsize=(20, 10)
)
plt.title(title, fontsize=20)
plt.xlabel('Event', fontsize=20)
plt.ylabel('Mean Order', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

file = os.path.join(path, f'{title}.png')
plt.savefig(file, dpi=100)

In [ ]:
# 每个次序做得最多的事
order_most_event = daily_event_order[
    (daily_event_order['date'] < '2023-01-01') & 
    (daily_event_order['date'] >= '2022-01-01')
].groupby(['order', 'event'])['date'].agg('count').reset_index()

order_most_event = order_most_event.rename(
    columns={'date': 'count'}
)

In [ ]:
# 每个顺序做的最多的k件事
k = 1
order_mostk_event = pd.concat(
    [order_most_event[order_most_event['order'] == m].\
    sort_values(by='count', ascending=False)[:k] 
    for m in order_most_event['order'].unique()], axis=0
)
order_mostk_event

### 2.4.7) 各事项关于日期的透视表

In [ ]:
# 2.4.7) 各事项关于日期的透视表
part = '2.4.7'

# 日度时长总和透视
sum_pivot = part2.get_pivot(df, 'date', 'event', 'duration', 'sum')
sum_pivot

In [ ]:
# 接单工作每天总时长分析
print(
    'The mean time for 接单工作 in year 2022 every day is', sum_pivot['event_duration_pivot_sum_接单工作'][
                        sum_pivot.index >= '2022-01-01'
].mean()
)

sum_pivot['event_duration_pivot_sum_接单工作'][
    sum_pivot['event_duration_pivot_sum_接单工作'] > 0
]

In [ ]:
title = f'{part} 每日接单工作时长折线图'

sum_pivot['event_duration_pivot_sum_接单工作'][
    sum_pivot['event_duration_pivot_sum_接单工作'] > 0
].plot(figsize=(20, 10), title=title)

plt.title(title, fontsize=20)
plt.xlabel('Date', fontsize=20)
plt.ylabel('Sum of duration', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

file = os.path.join(path, f'{title}.png')
plt.savefig(file, dpi=100)

In [ ]:
title = f'{part} 每日接单工作时长直方图'

sum_pivot['event_duration_pivot_sum_接单工作'][
    sum_pivot['event_duration_pivot_sum_接单工作'] > 0
].plot(
    kind='hist', figsize=(20, 10), title=title
)

plt.title(title, fontsize=20)
plt.xlabel('Sum of duration', fontsize=20)
plt.ylabel('Frequency', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

file = os.path.join(path, f'{title}.png')
plt.savefig(file, dpi=100)

In [ ]:
# 磨蹭每天总时长分析
print(
    'The mean time for 磨蹭 in year 2022 every day is', sum_pivot['event_duration_pivot_sum_磨蹭'][
                    sum_pivot.index >= '2022-01-01'
].mean()
)

sum_pivot['event_duration_pivot_sum_磨蹭'][
    sum_pivot['event_duration_pivot_sum_磨蹭'] > 0
]

In [ ]:
title = f'{part} 每日磨蹭时长折线图'

sum_pivot['event_duration_pivot_sum_磨蹭'][
    sum_pivot['event_duration_pivot_sum_磨蹭'] > 0
].plot(figsize=(20, 10), title=title)


plt.title(title, fontsize=20)
plt.xlabel('Date', fontsize=20)
plt.ylabel('Sum of duration', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

file = os.path.join(path, f'{title}.png')
plt.savefig(file, dpi=100)

In [ ]:
title = f'{part} 每日磨蹭时长直方图'

sum_pivot['event_duration_pivot_sum_磨蹭'][
    sum_pivot['event_duration_pivot_sum_磨蹭'] > 0
].plot(
    kind='hist', figsize=(20, 10), title=title
)

plt.title(title, fontsize=20)
plt.xlabel('Count', fontsize=20)
plt.ylabel('Frequency', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

file = os.path.join(path, f'{title}.png')
plt.savefig(file, dpi=100)

In [ ]:
# 次数透视
count_pivot = part2.get_pivot(df, 'date', 'event', 'duration', 'count')
count_pivot

In [ ]:
# 接单工作次数分析
title = f'{part} 每日接单工作次数折线图'

count_pivot['event_duration_pivot_count_接单工作'][
    count_pivot['event_duration_pivot_count_接单工作'] > 0
].plot(figsize=(20, 10), title=title)

plt.title(title, fontsize=20)
plt.xlabel('Date', fontsize=20)
plt.ylabel('Count', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

file = os.path.join(path, f'{title}.png')
plt.savefig(file, dpi=100)

In [ ]:
title = f'{part} 每日接单工作次数直方图'

count_pivot['event_duration_pivot_count_接单工作'][
    count_pivot['event_duration_pivot_count_接单工作'] > 0
].plot(kind='hist', figsize=(20, 10), title=title)

plt.title(title, fontsize=20)
plt.xlabel('Count', fontsize=20)
plt.ylabel('Frequency', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

file = os.path.join(path, f'{title}.png')
plt.savefig(file, dpi=100)

In [ ]:
# 磨蹭次数分析
print(
    'The mean count for 磨蹭 in year 2022 every day is', count_pivot['event_duration_pivot_count_磨蹭'][
                    count_pivot.index >= '2022-01-01'
].mean()
)

count_pivot['event_duration_pivot_count_磨蹭'][
    count_pivot['event_duration_pivot_count_磨蹭'] > 0
]

In [ ]:
title = f'{part} 每日磨蹭次数折线图'

count_pivot['event_duration_pivot_count_磨蹭'][
    count_pivot['event_duration_pivot_count_磨蹭'] > 0
].plot(figsize=(20, 10), title=title)

plt.title(title, fontsize=20)
plt.xlabel('Date', fontsize=20)
plt.ylabel('Count', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

file = os.path.join(path, f'{title}.png')
plt.savefig(file, dpi=100)

In [ ]:
title = f'{part} 每日磨蹭次数直方图'

count_pivot['event_duration_pivot_count_磨蹭'][
    count_pivot['event_duration_pivot_count_磨蹭'] > 0
].plot(kind='hist', figsize=(20, 10), title=title)

plt.title(title, fontsize=20)
plt.xlabel('Count', fontsize=20)
plt.ylabel('Frequency', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

file = os.path.join(path, f'{title}.png')
plt.savefig(file, dpi=100)

### 2.4.8) 每年, 月, 星期的事项关键词词云图

In [ ]:
part = '2.4.8'
# 年度关键词词云图
col = 'year'
year_key_words = part2.get_key_words(df, col)
part2.plot_key_word_cloud(year_key_words, col, part, path=path, mask=1)
year_key_words

In [ ]:
# 年度关键词词云图
part2.plot_key_word_cloud(year_key_words, col, part, path=path, mask=None)

In [ ]:
# 2022年每月关键词
col = 'month'
month_key_words = part2.get_key_words(df[df['year'] == '2022'], col)
part2.plot_key_word_cloud(month_key_words, col, part, path=path)
month_key_words

In [ ]:
# 2022年 每day_period关键词
col = 'day_period'
day_period_key_words = part2.get_key_words(df[df['year'] == '2022'], col)
part2.plot_key_word_cloud(day_period_key_words, col, part, path=path)
day_period_key_words

In [ ]:
# 2022年 duration_attr关键词
col = 'duration_attr'
duration_attr_key_words = part2.get_key_words(df[df['year'] == '2022'], col)
part2.plot_key_word_cloud(duration_attr_key_words, col, part, path=path)
duration_attr_key_words

In [ ]:
# 2022年 attr关键词
col = 'attr'
attr_key_words = part2.get_key_words(df[df['year'] == '2022'], col, threshold=.8)
part2.plot_key_word_cloud(attr_key_words, col, part, path=path)
attr_key_words

In [ ]:
# 2022年 weekday关键词
col = 'weekday'
weekday_key_words = part2.get_key_words(df[df['year'] == '2022'], col, threshold=1)
part2.plot_key_word_cloud(weekday_key_words, col, part, path=path)
weekday_key_words

In [ ]:
# 2022年 mid_hour关键词
col = 'mid_hour'
midhour_key_words = part2.get_key_words(df[df['year'] == '2022'], col, threshold=1)
part2.plot_key_word_cloud(midhour_key_words, col, part, path=path)
midhour_key_words

### 2.4.9) 每天事项的对应先后以及月度关联的网络图

In [ ]:
# 2.4.8) 每天事项的对应先后以及月度关联的网络图
part = '2.4.9'

In [ ]:
part2.month2event(df, 2019, event2attr, part, path)

In [ ]:
part2.month2event(df, 2020, event2attr, part, path)

In [ ]:
part2.month2event(df, 2021, event2attr, part, path)

In [ ]:
part2.month2event(df, 2022, event2attr, part, path)

In [ ]:
part2.PostEvent(df, 2019, event2attr, part, path)

In [ ]:
part2.PostEvent(df, 2020, event2attr, part, path)

In [ ]:
part2.PostEvent(df, 2021, event2attr, part, path)

In [ ]:
part2.PostEvent(df, 2022, event2attr, part, path)

## 2.5 一键式EDA

### 2.5.1 原始数据

In [ ]:
AV = AutoViz_Class()
dft = AV.AutoViz(
    filename=None,  
    sep=",",  
    depVar="duration",  
    dfte=df,  
    header=0,
    verbose=0,  
    lowess=True,  
    chart_format="svg",  
    max_rows_analyzed=150000,  
    max_cols_analyzed=30,
)

### 2.5.2 衍生数据

In [ ]:
AV = AutoViz_Class()
dft = AV.AutoViz(
    filename=None,  
    sep=",",  
    depVar="effective time",  
    dfte=daily_attr_copy,  
    header=0,
    verbose=0,  
    lowess=True,  
    chart_format="svg",  
    max_rows_analyzed=150000,  
    max_cols_analyzed=30,
)

# 3.数据分析

## 3.1 方差分析每月或是每年的事项是否有差异

### 3.1.1 原始事项时长数据的方差分析

In [ ]:
# 原始事项数据的方差分析
part = '3.1.1'

def vr_event(data, num, cate):
    formula = f'{num}~C({cate})'
    anova_re_2019 = anova_lm(ols(formula, data=data[[num, cate]]).fit())
    return anova_re_2019.loc[f'C({cate})', 'PR(>F)']

In [ ]:
# 事项时长数据
vr_duration_table = pd.DataFrame()
for event in tqdm(df['event'].unique()):
    for cate in cate_vars:
        if cate not in ['event', 'attr']:
            temp = df[df['event'] == event].copy()
            temp[cate] = temp[cate].astype(str)
            res = vr_event(temp, 'duration', cate)
            vr_duration_table.loc[event, cate] = res

# 该表格是各个事项的时长数据关于各分类变量的方差分析结果
vr_duration_table

In [ ]:
# 选取那些在不同分类情况下时长有显著差异的数据
vr_duration_table.to_excel('事项时长数据的方差分析结果.xlsx', index=True)
significant_event = vr_duration_table[vr_duration_table > .05].dropna(axis=0, how='all')
significant_event

In [ ]:
# 对于有显著差异的事项 做出在不同分类中均值的bar图
mins = 30  # 剔除掉出现次数少的事项
for e in tqdm(significant_event.index):
    if len(df[df['event'] == e]) >= mins:
        for c in significant_event.columns:
            if significant_event.loc[e, c] >= .05:
                plt.figure(dpi=100)
                df[df['event'] == e].groupby(c)['duration'].agg('mean').plot(
                    kind='bar', figsize=(20, 10)
                )
                title = f'{part} mean value of {e} duration in different {c} groups'
                plt.title(title, fontsize=20)
                plt.xlabel(e, fontsize=20)
                plt.ylabel('Mean value', fontsize=20)
                plt.xticks(fontsize=20)
                plt.yticks(fontsize=20)
                file = os.path.join(path, f'{title}.png')
                plt.savefig(file, dpi=100) 

In [ ]:
# 只分析2022年
vr_duration_table2022 = pd.DataFrame()
for event in tqdm(df['event'].unique()):
    for cate in cate_vars:
        if cate not in ['event', 'attr', 'year']:
            try:
                temp = df[
                    (df['event'] == event) & (df['year'] == '2022')
                ].copy()
                temp[cate] = temp[cate].astype(str)
                res = vr_event(temp, 'duration', cate)
                vr_duration_table2022.loc[event, cate] = res
            except:
                continue

# 该表格是各个事项的时长数据关于各分类变量的方差分析结果
vr_duration_table2022

In [ ]:
significant_event2022 = vr_duration_table2022[vr_duration_table2022 > .05].\
    dropna(axis=0, how='all')

mins = 20

for e in tqdm(significant_event2022.index):
    if len(df[df['event'] == e]) >= mins:
        for c in significant_event2022.columns:
            if significant_event2022.loc[e, c] >= .05:
                plt.figure(dpi=100)
                df[
                    (df['event'] == e) & (df['year'] == '2022')
                ].groupby(c)['duration'].agg('mean').plot(
                    kind='bar', figsize=(20, 10)
                )
                title = f'{part} mean value of {e} duration in different {c} groups in year 2022'
                plt.title(title, fontsize=20)
                plt.xlabel(e, fontsize=20)
                plt.ylabel('Mean value', fontsize=20)
                plt.xticks(fontsize=20)
                plt.yticks(fontsize=20)
                file = os.path.join(path, f'{title}.png')
                plt.savefig(file, dpi=100) 

### 3.1.2 衍生数据的方差分析

In [ ]:
part = '3.1.2'

daily_attr['weekday'] = pd.to_datetime(daily_attr['date']).dt.weekday + 1
daily_attr['year'] = daily_attr['date'].apply(lambda x:str(x).split('-')[0])
daily_attr['month'] = daily_attr['date'].apply(lambda x:str(x).split('-')[1])
daily_attr['day'] = daily_attr['date'].apply(lambda x:str(x).split('-')[2])
daily_attr['week_order'] = daily_attr['date'].map(lambda x: pd.Period(x, freq='W'))
daily_attr

In [ ]:
# 变量类型划分
num_vars = [
    '有效时间', '浪费时间', '必要时间', '手机时间'
]

cates_vars = [
    'year', 'month', 'day', 'weekday', 'week_order'
]

# 逐一进行方差分析
daily_sum_table = pd.DataFrame()
for num in tqdm(num_vars):
    for cate in cates_vars:
        temp = daily_attr[[num, cate]].copy()
        temp[cate] = temp[cate].astype(str)
        res = vr_event(temp, num, cate)
        daily_sum_table.loc[num, cate] = res

# 该表格是各个事项的时长日度求和数据数据关于各分类变量的方差分析结果
daily_sum_table

In [ ]:
# 对于有显著差异的事项 做出在不同分类中均值的bar图
for e in tqdm(daily_sum_table.index):
    for c in daily_sum_table.columns:
        if daily_sum_table.loc[e, c] >= .05:
            plt.figure(dpi=100)
            daily_attr.groupby(c)[e].agg('mean').plot(
                kind='bar', figsize=(20, 10)
            )
            title = f'{part} mean value of {e} duration in different {c} groups'
            plt.title(title, fontsize=20)
            plt.xlabel(e, fontsize=20)
            plt.ylabel('Mean value', fontsize=20)
            plt.xticks(fontsize=20)
            plt.yticks(fontsize=20)
            file = os.path.join(path, f'{title}.png')
            plt.savefig(file, dpi=100) 

## 3.2 LSTM预测前后事项

### 3.2.1 建立字符索引

In [ ]:
corpus_chars = df['event'].values
idx_to_char = list(set(corpus_chars)) # 去重，得到索引到字符的映射
char_to_idx = {char: i for i, char in enumerate(idx_to_char)} # 字符到索引的映射
vocab_size = len(char_to_idx)
print(vocab_size)
print('len(corpus_chars) = ', len(corpus_chars))

corpus_indices = [char_to_idx[char] for char in corpus_chars]  # 将每个字符转化为索引，得到一个索引的序列
sample = corpus_indices[: 20]
print('chars:', ' '.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)

In [ ]:
idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)
corpus_indices = [char_to_idx[char] for char in corpus_chars]

### 3.2.2 建立LSTM模型

In [ ]:
num_inputs, num_hiddens, num_outputs = vocab_size, 128, vocab_size
pred_period, pred_len, prefixes = 50, 50, ['磨蹭', '运动']
rnn_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens)
num_steps, batch_size = 35, 2
X = torch.rand(num_steps, batch_size, vocab_size)
state = None
Y, state_new = rnn_layer(X, state)
print(Y, state_new)

In [ ]:
device = 'cpu'
model = part3.RNNModel(rnn_layer, vocab_size).to(device)
part3.predict_rnn_pytorch(['运动'], 10, model, vocab_size, device, idx_to_char, char_to_idx)

### 3.2.3 模型训练

In [ ]:
rnn_layer = nn.LSTM(input_size=vocab_size, hidden_size=num_hiddens)
model = part3.RNNModel(rnn_layer, vocab_size).to(device)
num_epochs, batch_size, lr, clipping_theta = 100, 64, 1e-2, 1e-2
print_num = 10
pred_period = num_epochs / print_num
pred_len, prefixes = 15, [
    ['华尔街日报'], 
    ['晚饭'], 
    ['摄影'], 
    ['接单工作'], 
    ['gis'],
    ['量化'], 
    ['玩游戏'],
]
part3.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
    corpus_indices, idx_to_char, char_to_idx,
    num_epochs, num_steps, lr, clipping_theta,
    batch_size, pred_period, pred_len, prefixes
)

In [ ]:
rnn_layer = nn.GRU(input_size=vocab_size, hidden_size=num_hiddens)
model = part3.RNNModel(rnn_layer, vocab_size).to(device)
num_epochs, batch_size, lr, clipping_theta = 100, 64, 1e-2, 1e-2
print_num = 10
pred_period = num_epochs / print_num
pred_len, prefixes = 15, [
    ['华尔街日报'], 
    ['晚饭'], 
    ['接单工作'], 
    ['托福听力'], 
    ['磨蹭'],
    ['写日记']
]
part3.train_and_predict_rnn_pytorch(model, num_hiddens, vocab_size, device,
    corpus_indices, idx_to_char, char_to_idx,
    num_epochs, num_steps, lr, clipping_theta,
    batch_size, pred_period, pred_len, prefixes
)

## 3.3 时长数据的简单聚类

In [ ]:
part = '3.3'

dates = pd.DataFrame(
    pd.date_range(
    start='2022-01-01',
    end='2022-12-31'
).astype(str), columns=['date']
)

data = pd.merge(
    dates, daily_attr[num_vars + ['date']], on='date', how='left'
)

In [ ]:
mergings = linkage(data.drop('date', axis=1).dropna())
plt.figure(figsize=(20, 10))
dendrogram(
    mergings,
    leaf_rotation=90,
    leaf_font_size=10, 
)
plt.yticks(fontsize=20)
title = f'{part} 2022时长数据层次聚类图'
plt.title(title, fontsize=10)
plt.savefig(os.path.join(path, f'{title}.png'), dpi=100)

## 3.4 关于日期的缺失值处理

In [ ]:
# 对日度数据做处理 只分析2022年
part = '3.4'

# 生成新特征
data['weekday'] = pd.to_datetime(data['date']).dt.weekday + 1
data['month'] = data['date'].apply(lambda x:str(x).split('-')[1])
data['day'] = data['date'].apply(lambda x:str(x).split('-')[2])
data['week_order'] = data['date'].map(lambda x: pd.Period(x, freq='W'))

data.set_index('date')[num_vars].plot(
    figsize=(20, 10)
)

title = '各类时长每日总和的折线图'
plt.title(title, fontsize=20)
plt.xlabel('Date', fontsize=20)
plt.ylabel('Sum of duration', fontsize=20)
plt.yticks(fontsize=20)
plt.xticks(fontsize=20)
plt.legend(loc=1, fontsize=20)

print('缺失的行数有', data['有效时间'].isnull().sum())

In [ ]:
# 采样三次样条处理缺失
for col in data.columns:
    if '时间' in col:
        upper, lower = data[col].max(), data[col].min()
        data[col] = data[col].interpolate(method='spline', order=3)
        # 避免插值结果太超过想象
        data[col] = np.where(abs(data[col]) > upper, upper, data[col])
        data[col] = np.where(data[col] < lower, lower, data[col])
data = data.fillna(axis=0, method='bfill') 


title = f'{part} 处理缺失后的每日时长总和图'
data.set_index('date')[num_vars].plot(
    figsize=(20, 10), title=title
)

plt.title(title, fontsize=20)
plt.xlabel('Date', fontsize=20)
plt.ylabel('Sum of duration', fontsize=20)
plt.yticks(fontsize=20)
plt.xticks(fontsize=20)
plt.legend(loc=1, fontsize=20)

file = os.path.join(path, f'{title}.png')
plt.savefig(file, dpi=100) 

print('缺失的行数有', data['有效时间'].isnull().sum())

## 3.5 关于日期的傅里叶分解求周期

In [ ]:
part = '3.5'

In [ ]:
# 有效时间
part3.FRT(data, '有效时间', part, path)

In [ ]:
# 浪费时间
part3.FRT(data, '浪费时间', part, path)

In [ ]:
# 必要时间
part3.FRT(data, '必要时间', part, path)

In [ ]:
# 手机时间
part3.FRT(data, '手机时间', part, path)

## 3.6 关于日期的传统时间序列分析

### 3.6.1 ARIMA分析

In [ ]:
part = '3.6.1'

In [ ]:
# 4类时间的ARIMA分析
arima_dict = {}
for col in num_vars:
    try:
        arima_dict[col] = part3.ARIMA_auto(data.set_index('date')[col], col, part, path)
    except:
        continue

### 3.6.2 GARCH分析

In [ ]:
part = '3.6.2'

In [ ]:
# 检验是否存在GARCH效应
arch_p_dict = {
    k: v.arch_p for k, v in arima_dict.items() 
}
print(arch_p_dict)

In [ ]:
# 对存在GARCH效应的序列做GARCH建模
for name, p in arch_p_dict.items():
    if p < 0.05:
        try:
            part3.garch_auto(
                data.set_index('date')[name], 
                name, 
                arima_dict[name].arima_order,
                part,
                path
            ) 
        except:
            continue

### 3.6.3 VAR分析

In [ ]:
part = '3.6.3'

In [ ]:
price_vol_int_var = part3.VAR(data[list(arima_dict.keys())], arima_dict, part, path)
price_vol_int_var.impulse_responses()
price_vol_int_var.variance_decompositon()

## 3.7 关于日期的NeuralProphet 时间序列

In [ ]:
part = '3.7'
NP_dict = {}
test_length = 30

for col in num_vars:
    part3.MyNeuralProphet(data, col, NP_dict, test_length, part, path)

## 3.8 关联规则

In [ ]:
part = '3.8'
te = TransactionEncoder()
df_tf = te.fit_transform(df.groupby('date')['event'].agg(list))
dff = pd.DataFrame(df_tf,columns=te.columns_)
# use_colnames=True表示使用元素名字，默认的False使用列名代表元素, 设置最小支持度min_support
frequent_itemsets = apriori(dff, min_support=0.05, use_colnames=True)
frequent_itemsets.sort_values(by='support', ascending=False, inplace=True)
# 选择2频繁项集
nwdf = frequent_itemsets[frequent_itemsets.itemsets.apply(lambda x: len(x)) == 2]
nwdf

In [ ]:
nwdf['from'] = nwdf['itemsets'].map(lambda x:list(x)[0])
nwdf['to'] = nwdf['itemsets'].map(lambda x:list(x)[1])
nwdf['weight'] = nwdf['support']
part2.net(nwdf, 'from', 'to', '2022', event2attr, part, path, weight='weight')

In [ ]:
# metric可以有很多的度量选项，返回的表列名都可以作为参数
association_rule = association_rules(frequent_itemsets,metric='confidence',min_threshold=0.9)

#关联规则可以提升度排序
association_rule.sort_values(by='lift',ascending=False,inplace=True)
association_rule.to_csv('association_rule.csv', index=False, encoding='gbk')
association_rule
# 规则是：antecedents->consequents

# 4.数据挖掘

## 4.1 通过生成聚合特征做日度时长数据的特征工程

In [ ]:
nums = [
    '浪费时间', '必要时间', '手机时间'
]

cates = [
    'month', 'day', 'weekday'
]

y = '有效时间'

In [ ]:
# 基于A/B的统计特征
for num_var in tqdm(nums):
    for cate_var in cates:
        for sts in ['mean', 'median', 'std', 'max', 'min']:
            data[f'{cate_var}_{num_var}_{sts}'] = data.groupby(cate_var)[num_var].\
                transform(sts).values
        data[f'{cate_var}_{num_var}_cv'] = data[f'{cate_var}_{num_var}_std'] / \
            data[f'{cate_var}_{num_var}_mean']
        for fun in funcs.all_functions:
            try:
                fun_name = fun.__name__
            except:
                fun_name = str(fun)
            data[f'{num_var}_{fun_name}_{cate_var}'] = data.groupby(cate_var)[num_var].\
                transform(fun).values

In [ ]:
# 流量平滑特征
# 基于A/B的统计特征
for num_var in tqdm(nums):
    for cate_var in cates:
        data[f'{num_var}_div_{cate_var}_{num_var}_mean'] = data[num_var] / \
            (data[f'{cate_var}_{num_var}_mean'] + 1e-5)
        data[f'{num_var}_div_{cate_var}_{num_var}_median'] = data[num_var] / \
            (data[f'{cate_var}_{num_var}_median'] + 1e-5)
        # 黄金组合特征
        data[f'{num_var}_minus_{cate_var}_{num_var}_mean'] = data[num_var] - \
            data[f'{cate_var}_{num_var}_mean']

In [ ]:
# 相邻有序类别统计特征的变化
sts_list = ['max', 'min', 'mean', 'std', 'median']
for num_var in tqdm(nums):
    for cate_var in cates:
        for sts in sts_list:
            data[f'{cate_var}_{num_var}_{sts}_diff']  = data[f'{cate_var}_{num_var}_{sts}'] - \
                data.groupby(cate_var)[f'{cate_var}_{num_var}_{sts}'].shift()        
            data[f'{cate_var}_{num_var}_{sts}_ratio'] = data[f'{cate_var}_{num_var}_{sts}'] / \
                data.groupby(cate_var)[f'{cate_var}_{num_var}_{sts}'].shift() 
            data[f'{cate_var}_{num_var}_{sts}_diff'].fillna(method='bfill', inplace=True)
            data[f'{cate_var}_{num_var}_{sts}_ratio'].fillna(method='bfill', inplace=True)

In [ ]:
# 事项序列的tf-idf特征
vec = TfidfVectorizer(ngram_range=(1, 3), min_df=.01)
vec_features = vec.fit_transform(
    df.groupby('date')['event'].agg(lambda x: ' '.join(x))
)
vec_features = pd.DataFrame(
    vec_features.toarray(), 
    columns=vec.get_feature_names()
)
data = pd.concat(
    [data, vec_features], axis=1
)

## 4.2 类别变量的one-hot encoding

In [ ]:
for cate_var in cates:
    onehot = pd.get_dummies(data[cate_var], drop_first=True, prefix=f'{cate_var}_')
    data = pd.concat([data.drop(cate_var, axis=1), onehot], axis=1)

data.to_excel('data_FE.xlsx', index=False)
data

## 4.3 利用boruta筛选特征

In [ ]:
# 划分数据集
data = pd.read_excel('data_FE.xlsx')
data = data[data['date'] < '2022-12-31'].reset_index()

# 处理缺失
for col in data.columns:
    if data[col].isnull().any():
        data[col] = data[col].fillna(data[col].mean())
data = data.dropna(axis=1, how='all')

train_size = .8
rows = int(data.shape[0] * train_size)
train, test = data.iloc[:(rows + 1), :], data.iloc[(rows + 1):, :]
X = train.drop(['date', 'week_order', y], axis=1)
Y = train[y]

In [ ]:
clf = XGBRegressor()
boruta = BorutaPy(
   estimator = clf,
   n_estimators = 'auto',
   max_iter = 100 # number of trials to perform
)

# 模型训练
boruta.fit(np.array(X), np.array(Y))
# 输出结果
green_area = X.columns[boruta.support_].to_list()
blue_area  = X.columns[boruta.support_weak_].to_list()
print('features in the green area:', green_area)
print('features in the blue area:', blue_area)
cols = green_area + blue_area
train = train[cols]

## 4.4 利用PCA降维

In [ ]:
dt = data[cols]
dt

In [ ]:
X_train = data.loc[train.index, cols]# dt.loc[train.index, :]
X_test = data.loc[test.index, cols]#dt.loc[test.index, :]
y_train = data.loc[train.index, y]#data.loc[train.index, y]
y_test = data.loc[test.index, y]#data.loc[test.index, y]

## 4.5 利用optuna优化参数

In [ ]:
sampler = TPESampler(seed=10) # for reproducibility

def objective(trial):
    dtrain = lgb.Dataset(X_train, label=y_train)
    
    param = {
        'objective': 'regression_l2',
        'metric': 'mse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-8, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 700, 3000),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    gbm = lgb.train(param, dtrain)
    return mean_squared_error(y_test, gbm.predict(X_test))

study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=100)
lgb_params = study.best_params
print(lgb_params)

In [ ]:
def objective(trial):
    dtrain = xgb.DMatrix(X_train, label=y_train)

    param = {
        'objective': 'reg:squarederror',
        'metric': 'mse',
        'verbosity': 0,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 512),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-8, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 700, 3000),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }

    gbm = xgb.train(param, dtrain)
    return mean_squared_error(y_test, gbm.predict(xgb.DMatrix(X_test, label=np.ones_like(y_test))))

study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=100)
xgb_params = study.best_params
print(xgb_params)

## 4.6 stacking 融合

In [ ]:
part = '4.6'

### 4.6.1 sklearn中的stacking

In [ ]:
FOLDS = 5
models = []
mses = []

gkf = part4.PurgedGroupTimeSeriesSplit(n_splits = FOLDS, group_gap=14)
splits = list(gkf.split(X_train, groups=X_train.index))
for train_indices, test_indices in splits:
        model = StackingRegressor(
            estimators=[
                ('rf', RandomForestRegressor()),
                ('ada', AdaBoostRegressor()),
                ('gb', GradientBoostingRegressor()),
                ('lgb', LGBMRegressor(**lgb_params)),
                ('xgb', XGBRegressor(**xgb_params))
            ],
            final_estimator=LGBMRegressor(**lgb_params),
            cv=2
        )
        model.fit(
            X_train,
            y_train
        )
        models.append(model)
        mses.append(mean_squared_error(
            model.predict(X_test), y_test
        ))

In [ ]:
print(mses, np.mean(mses))

In [ ]:
pred = pd.DataFrame([model.predict(X_test) for model in models]).mean()
title = f'{part} Predicted value VS real value use sklearn'
plt.figure(figsize=(20, 10))
plt.scatter(y_test, pred, s=600)
plt.xlabel('real value', fontsize=20)
plt.ylabel('predicted value', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.title(title, fontsize=20)
file = os.path.join(path, f'{title}.png')
plt.savefig(file, dpi=100) 

In [ ]:
mean_squared_error(pred, y_test)

### 4.6.2 自己实现stacking

In [ ]:
import importlib
from utils import part4
importlib.reload(part4)

In [ ]:
clf_list = [part4.rf_reg, part4.ada_reg, part4.gb_reg, part4.xgb_reg, part4.lgb_reg]
train_y = y_train.values
pred = part4.stacking_pred(
    X_train.values,
    train_y, 
    X_test.values, 
    gkf,
    clf_list, 
    label_split=range(len(train_y)),
    clf_fin='lgb'
)

In [ ]:
title = f'{part} Predicted value VS real value'
plt.figure(figsize=(20, 10))
plt.scatter(y_test, pred.reshape(1, -1)[0], s=600)
plt.xlabel('real value', fontsize=20)
plt.ylabel('predicted value', fontsize=20)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.title(title, fontsize=20)
file = os.path.join(path, f'{title}.png')
plt.savefig(file, dpi=100) 

In [ ]:
preds = pd.concat(
    [y_test.reset_index(), 
    pd.Series(pred.reshape(1, -1)[0], name='预测有效时间').reset_index()], 
    axis=1
)[['有效时间', '预测有效时间']]

preds

In [ ]:
mean_squared_error(pred, y_test)

# 更多学习笔记，请关注公众号【杰然不同之GR】 #
![公众号二维码](http://m.qpic.cn/psc?/V10twqic2oh0r6/TmEUgtj9EK6.7V8ajmQrEG6xI.X7icgy*l8zd9O9qB3X6.AQyIe0uOSHtI7ti9nULpRDinQuLz61UqAz2Qxai8Ay.JgGp*bQUKKMa4c9AyU!/b&bo=rgGuAa4BrgEBGT4!&rf=viewer_4&t=5)